In [5]:
from dataset import ERADataset
import xarray as xr
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import torch

In [6]:
root_dir = "./data/era_5_data"

dataset = ERADataset(root_dir, 0, [2003,2004])

In [7]:
dataset.dataset[]

In [16]:
geo = dataset.data_vars['geopotential_500']
temp = dataset.data_vars['temperature_850']